# Thinking about flows
## A single transition flow
Taking a step back from the simple model presented in the previous notebook, 
this notebook digs into what we mean by a transition flow.
To do this, let's consider a really simple model without even an infection process,
and only a single transition flow between two compartments.
![](../images/source_dest_structure.svg)

In [ ]:
# If running on Google Colab, run the following line of code to install the summer package
# %pip install summerepi2

In [ ]:
import pandas as pd
import numpy as np
pd.options.plotting.backend = "plotly"

from summer2 import CompartmentalModel
from summer2.parameters import Parameter

In [ ]:
def get_single_transition_model(
    model_config: dict
) -> CompartmentalModel:
    """
    Generate an instance of an extremely simple two-compartment model that allows transition
    from one of the two states to the other through a single flow linking them.
    
    Args:
        model_config: Non-parameter values needed to define the model structure    
    Returns:
        The summer model object
    """
    compartments = (
        "source",
        "destination",
    )
    analysis_times = (
        model_config["start_time"], 
        model_config["end_time"],
    )
    
    model = CompartmentalModel(
        times=analysis_times,
        compartments=compartments,
        infectious_compartments=[],
    )
    model.set_initial_population(
        distribution={"source": model_config["population"]}
    )
    model.add_transition_flow(
        "transition", 
        fractional_rate=Parameter("transition_rate"), 
        source="source", 
        dest="destination"
    )
    return model

With this single transition flow, 
people in the source compartment transition to the destination compartment 
at a _per capita_ rate defined by the flow's parameter.
By "_per capita_" we mean that the rate of this flow is calculated as the product of the parameter value assigned to the flow rate
and the size of the population in the source compartment.
_summer_ will automatically multiply the rate associated with a particular flow
by the size of the compartment it is being applied to.

In [ ]:
config = {
    "population": 1.,
    "start_time": 0.,
    "end_time": 20.,
}
parameter = {
    "transition_rate": 0.1,
}

transition_model = get_single_transition_model(config)

transition_model.run(parameters=parameter)

transition_model.get_outputs_df().plot(
    labels={"index": "time", "value": "proportion"},
)

## Analytic equivalent
For a model as simple as this,
it is easy to recreate the same system analytically,
as shown below.
However, very quickly as we start to add compartments and complexity to our models,
we reach the point that we cannot exactly predict compartment sizes
without solving the system numerically,
which we will address in the next notebook.

In [ ]:
pd.DataFrame(
    {
        "source": np.exp([-parameter["transition_rate"] * i_time for i_time in range(21)]),
        "destination": 1. - np.exp([-parameter["transition_rate"] * i_time for i_time in range(21)]),
    }
).plot(
    labels={"index": "time", "value": "proportion"},
)

## Other flow types
In infectious diseases modelling, 
transition flows should be used to represent a person's progression from one state to another.
For example, a person may recover from active disease some period of time after being infected.
Of course, transition flows are not the only type of flow that we might want to include in our models.
_summer_ also supports a range of other flows that we may wish to use.
All of these are described in more detail in the _summer_ documentation
and the epidemiological considerations associated with some of these will be discussed in later notebooks.
The following is a very brief list of the available flows
and whether each flow has a source/origin compartment
and a destination compartment.

|Flow name|Source compartment?|Destination compartment?|
|---|---|---|
|Infection density|Yes|Yes|
|Infection frequency flow|Yes|Yes|
|Death flow|Yes|No|
|Universal death flows|Yes|No|
|Importation flow|No|Yes|
|Crude birth flow|No|Yes|
|Replacement birth flow|No|Yes|

Also note that _summer_ does not restrict the user to using only fixed rates for flows
and functions of time or other emergent quantities from the model can be used to modify the flow rates.
Flow can also be adjusted after they are first applied,
including through the process of stratification.
This is described in more detail in the _summer_ documentation.